In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [44]:
df_customer = pd.read_csv("C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/data/cleaned/movie_customer_cleaned.csv", engine= "python", encoding='utf-8')
df_download = pd.read_csv("C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/data/cleaned/movie_download_cleaned.csv", engine= "python", encoding='utf-8')
df_price = pd.read_csv("C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/data/raw/movie_price_spreadsheets.csv", engine= "python", encoding='utf-8')
df_meta = pd.read_csv("C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/data/cleaned/movie_meta_cleaned_final5.csv", engine= "python", encoding='utf-8')

In [3]:
df_download.head(5)

,customer_id,item_id,down_year,down_date,weekday,inf_year,down_price
0,C5001,I-1010,2014,02-21-2014,6,1.051872,0.946685
1,C5001,I-1014,2014,04-04-2014,6,1.051872,0.946685
2,C5001,I-1017,2014,01-19-2014,1,1.051872,0.946685
3,C5001,I-1028,2014,01-21-2014,3,1.051872,0.946685
4,C5001,I-1035,2014,03-11-2014,3,1.051872,0.899351


In [4]:
df_download['down_price'].describe()

count    73085.000000
mean         0.974824
std          0.089535
min          0.810000
25%          0.918900
50%          0.931777
75%          1.021000
max          1.262247
Name: down_price, dtype: float64

In [5]:
# 고객별 매출
rev_customer = df_download.groupby(['customer_id'])['down_price'].sum()
df_customer = pd.merge(df_customer,rev_customer, on='customer_id',how='left')
df_customer = df_customer.rename(columns={'down_price':'customer_sales'})
# df_customer.drop(['down_price_x','down_price_y'],inplace=True,axis=1)

In [6]:
# 영화별 매출
rev_movie = df_download.groupby(['item_id'])['down_price'].sum()
df_meta = pd.merge(df_meta,rev_movie, on='item_id',how='left')
df_meta = df_meta.rename(columns={'down_price':'movie_down_sales'})

In [7]:
df_meta.columns

Index(['movie_id', 'title', 'release_year', 'release_date', 'runtime',
       'mpa_rating', 'imdb_score', 'dvd_sales', 'blu_sales', 'total_sales',
       'legs', 'share', 'inf_income_usa', 'theater_opening', 'theater_total',
       'series', 'genre', 'src', 'creative_type', 'country', 'director',
       'actor', 'writer', 'description', 'synop', 'contract_price',
       'studio_score', 'price_class', 'language', 'awards', 'poster',
       'metascore', 'release_dvd', 'big_awards_num', 'awards_win_num',
       'awards_nomin_num', 'reviews_users', 'reviews_critics', 'prd_mthd',
       'prd_company', 'budget', 'series_new', 'kwrds', 'actor_1', 'actor_2',
       'actor_3', 'actor_4', 'income_opening', 'votes', 'income_usa',
       'income_int', 'income_ww', 'item_id', 'inv_exist', 'contract_year',
       'inf', 'genre_action', 'genre_adventure', 'genre_drama', 'genre_sci-fi',
       'genre_fantasy', 'genre_thriller', 'genre_animation', 'genre_family',
       'genre_musical', 'genre_comedy',

In [8]:
rev_customer

customer_id
C5001    10.997324
C5002    23.661645
C5003    48.821681
C5004    38.822686
C5005    36.841977
           ...    
C8273     9.990000
C8274    21.770000
C8275     4.410000
C8276    13.480000
C8277     3.800000
Name: down_price, Length: 2738, dtype: float64

In [9]:
df_customer['customer_id']

0       C5001
1       C5002
2       C5003
3       C5004
4       C5005
        ...  
3272    C8273
3273    C8274
3274    C8275
3275    C8276
3276    C8277
Name: customer_id, Length: 3277, dtype: object

In [10]:
df_customer[df_customer['customer_id'] == '.']
len(rev_customer)
len(df_customer)
df_customer['customer_sales'].isna().sum()

539

In [11]:
df_customer['customer_sales'] = df_customer['customer_sales'].fillna('.')
df_meta['movie_down_sales'] = df_meta['movie_down_sales'].fillna('.')

In [12]:
df_meta['contract_year']

0            .
1            .
2       2015.0
3       2018.0
4       2014.0
         ...  
5148         .
5149         .
5150         .
5151         .
5152         .
Name: contract_year, Length: 5153, dtype: object

In [13]:
df_meta[df_meta['contract_year'] == '.']


,movie_id,title,release_year,release_date,runtime,mpa_rating,imdb_score,dvd_sales,blu_sales,total_sales,...,genre_adult,genre_news,genre_reality-tv,genre_film-noir,genre_talk-show,country_1,country_2,country_3,dvd_over_income,movie_down_sales
0,tt4154796,Avengers: Endgame,2019,26 Apr 2019,181.0,PG-13,8.5,19581271.0,69437591.0,89018862.0,...,0.0,0.0,0.0,0.0,0.0,USA,.,.,0.02281207703410988,.
1,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,20 May 2011,136.0,PG-13,6.6,35478573.0,58936309.0,94414882.0,...,0.0,0.0,0.0,0.0,0.0,USA,UK,.,0.14717498837185788,.
8,tt0401729,John Carter,2012,09 Mar 2012,132.0,PG-13,6.6,24755272.0,16426306.0,41181578.0,...,0.0,0.0,0.0,0.0,0.0,USA,.,.,0.33884094728841185,.
10,tt6105098,The Lion King,2019,19 Jul 2019,118.0,PG,6.9,16106589.0,31098360.0,47204949.0,...,0.0,0.0,0.0,0.0,0.0,USA,.,.,0.029627413326554115,.
11,tt0398286,Tangled,2010,24 Nov 2010,101.0,PG,7.7,179787688.0,65663692.0,245451380.0,...,0.0,0.0,0.0,0.0,0.0,USA,.,.,0.8952592111252229,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5148,tt5997666,Jab Harry Met Sejal,2017,04 Aug 2017,144.0,NOT RATED,5.3,.,.,.,...,0.0,0.0,0.0,0.0,0.0,India,.,.,.,.
5149,tt5970844,Thugs of Hindostan,2018,08 Nov 2018,164.0,NOT RATED,4.0,.,.,.,...,0.0,0.0,0.0,0.0,0.0,India,.,.,.,.
5150,tt5956100,Tiger Zinda Hai,2017,22 Dec 2017,145.0,NOT RATED,6.0,.,.,.,...,0.0,0.0,0.0,0.0,0.0,India,.,.,.,.
5151,tt5882970,Tubelight,2017,23 Jun 2017,170.0,NOT RATED,4.0,.,.,.,...,0.0,0.0,0.0,0.0,0.0,India,.,.,.,.


In [14]:
# inflation반영 contract_price
df_meta['contract_price_inf'] = '.'
for i in df_meta.index:
    for j in range(len(df_download['down_year'].unique())):
        if df_meta['contract_year'][i] != '.':
            if float(df_meta['contract_year'][i]) == df_download['down_year'].unique()[j]:
                df_meta['contract_price_inf'][i] = float(df_meta['contract_price'][i]) * df_download['inf_year'].unique()[j]
                print(i)


C:\Users\cresc\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


2
3
4
5
6
7
9
12
13
14
15
16
17
18
20
24
25
28
31
33
39
41
43
45
47
50
51
52
64
72
73
74
75
78
83
92
93
98
99
100
101
102
103
114
115
116
117
129
137
138
139
140
154
156
157
158
159
160
162
163
169
170
176
206
226
237
240
242
260
270
283
307
308
310
335
398
429
528
529
532
610
614
617
618
661
662
734
737
766
934
935
961
1053
1059
1335
1568
1569
1570
1580
1932
1952
2072
2699
3807
4626
4703


In [15]:
df_download['down_year'].unique()

array([2014, 2015, 2016, 2017, 2018], dtype=int64)

In [16]:
float(df_meta['contract_year'][12]) == float(df_download['down_year'].unique()[0])

True

In [17]:
df_meta['runtime'][12]

139.0

In [18]:
df_meta[df_meta['inv_exist'] == 1]

,movie_id,title,release_year,release_date,runtime,mpa_rating,imdb_score,dvd_sales,blu_sales,total_sales,...,genre_news,genre_reality-tv,genre_film-noir,genre_talk-show,country_1,country_2,country_3,dvd_over_income,movie_down_sales,contract_price_inf
2,tt2395427,Avengers: Age of Ultron,2015,01 May 2015,141.0,PG-13,7.3,33569045.0,55802159.0,89371204.0,...,0.0,0.0,0.0,0.0,USA,.,.,0.07313423932087944,1042.26,755.774
3,tt4154756,Avengers: Infinity War,2018,27 Apr 2018,156.0,PG-13,8.5,23700665.0,77736363.0,101437028.0,...,0.0,0.0,0.0,0.0,USA,.,.,0.034914738435503155,161,1000
4,tt0449088,Pirates of the Caribbean: At World's End,2007,25 May 2007,167.0,PG-13,7.1,314040375.0,7640482.0,321680857.0,...,0.0,0.0,0.0,0.0,USA,.,.,1.0149309794271015,1019.24,767.867
5,tt0974015,Justice League,2017,17 Nov 2017,121.0,PG-13,6.4,19449630.0,40475432.0,59925062.0,...,0.0,0.0,0.0,0.0,USA,Canada,UK,0.08492387237781913,375.43,745.33
6,tt2379713,Spectre,2015,06 Nov 2015,148.0,PG-13,6.8,12917812.0,24092452.0,37010264.0,...,0.0,0.0,0.0,0.0,UK,USA,Austria,0.064565114413192,472.11,755.774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2072,tt4846340,Hidden Figures,2016,06 Jan 2017,126.0,PG,7.8,18636040.0,11544851.0,30180891.0,...,0.0,0.0,0.0,0.0,USA,.,.,0.10987759034197629,590.648,1034.27
2699,tt6644200,A Quiet Place,2018,06 Apr 2018,95.0,PG-13,7.5,6628735.0,10373895.0,17002630.0,...,0.0,0.0,0.0,0.0,USA,.,.,0.035254660432006465,159,920
3807,tt5052448,Get Out,2017,24 Feb 2017,104.0,R,7.7,7363633.0,6449455.0,13813088.0,...,0.0,0.0,0.0,0.0,USA,Japan,.,0.04182915918887264,579.142,939.32
4626,tt0121766,Star Wars: Episode III - Revenge of the Sith,2005,19 May 2005,139.0,PG-13,7.5,.,.,.,...,0.0,0.0,0.0,0.0,USA,.,.,.,868.382,967.722


In [19]:
df_meta['contract_year'][2]

'2015.0'

In [20]:
type(df_download['down_year'].unique()[0])

numpy.int64

In [21]:
df_meta

,movie_id,title,release_year,release_date,runtime,mpa_rating,imdb_score,dvd_sales,blu_sales,total_sales,...,genre_news,genre_reality-tv,genre_film-noir,genre_talk-show,country_1,country_2,country_3,dvd_over_income,movie_down_sales,contract_price_inf
0,tt4154796,Avengers: Endgame,2019,26 Apr 2019,181.0,PG-13,8.5,19581271.0,69437591.0,89018862.0,...,0.0,0.0,0.0,0.0,USA,.,.,0.02281207703410988,.,.
1,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,20 May 2011,136.0,PG-13,6.6,35478573.0,58936309.0,94414882.0,...,0.0,0.0,0.0,0.0,USA,UK,.,0.14717498837185788,.,.
2,tt2395427,Avengers: Age of Ultron,2015,01 May 2015,141.0,PG-13,7.3,33569045.0,55802159.0,89371204.0,...,0.0,0.0,0.0,0.0,USA,.,.,0.07313423932087944,1042.26,755.774
3,tt4154756,Avengers: Infinity War,2018,27 Apr 2018,156.0,PG-13,8.5,23700665.0,77736363.0,101437028.0,...,0.0,0.0,0.0,0.0,USA,.,.,0.034914738435503155,161,1000
4,tt0449088,Pirates of the Caribbean: At World's End,2007,25 May 2007,167.0,PG-13,7.1,314040375.0,7640482.0,321680857.0,...,0.0,0.0,0.0,0.0,USA,.,.,1.0149309794271015,1019.24,767.867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5148,tt5997666,Jab Harry Met Sejal,2017,04 Aug 2017,144.0,NOT RATED,5.3,.,.,.,...,0.0,0.0,0.0,0.0,India,.,.,.,.,.
5149,tt5970844,Thugs of Hindostan,2018,08 Nov 2018,164.0,NOT RATED,4.0,.,.,.,...,0.0,0.0,0.0,0.0,India,.,.,.,.,.
5150,tt5956100,Tiger Zinda Hai,2017,22 Dec 2017,145.0,NOT RATED,6.0,.,.,.,...,0.0,0.0,0.0,0.0,India,.,.,.,.,.
5151,tt5882970,Tubelight,2017,23 Jun 2017,170.0,NOT RATED,4.0,.,.,.,...,0.0,0.0,0.0,0.0,India,.,.,.,.,.


In [22]:
df_download

,customer_id,item_id,down_year,down_date,weekday,inf_year,down_price
0,C5001,I-1010,2014,02-21-2014,6,1.051872,0.946685
1,C5001,I-1014,2014,04-04-2014,6,1.051872,0.946685
2,C5001,I-1017,2014,01-19-2014,1,1.051872,0.946685
3,C5001,I-1028,2014,01-21-2014,3,1.051872,0.946685
4,C5001,I-1035,2014,03-11-2014,3,1.051872,0.899351
...,...,...,...,...,...,...,...
73080,C8276,I-1101,2018,06-01-2018,6,1.000000,1.000000
73081,C8277,I-1017,2018,06-15-2018,6,1.000000,0.900000
73082,C8277,I-1078,2018,04-19-2018,5,1.000000,0.900000
73083,C8277,I-1099,2018,07-07-2018,7,1.000000,1.000000


In [23]:
df_meta['contract_price_inf']

0             .
1             .
2       755.774
3          1000
4       767.867
         ...   
5148          .
5149          .
5150          .
5151          .
5152          .
Name: contract_price_inf, Length: 5153, dtype: object

In [24]:
df_meta[df_meta['movie_down_sales'] != '.']

,movie_id,title,release_year,release_date,runtime,mpa_rating,imdb_score,dvd_sales,blu_sales,total_sales,...,genre_news,genre_reality-tv,genre_film-noir,genre_talk-show,country_1,country_2,country_3,dvd_over_income,movie_down_sales,contract_price_inf
2,tt2395427,Avengers: Age of Ultron,2015,01 May 2015,141.0,PG-13,7.3,33569045.0,55802159.0,89371204.0,...,0.0,0.0,0.0,0.0,USA,.,.,0.07313423932087944,1042.26,755.774
3,tt4154756,Avengers: Infinity War,2018,27 Apr 2018,156.0,PG-13,8.5,23700665.0,77736363.0,101437028.0,...,0.0,0.0,0.0,0.0,USA,.,.,0.034914738435503155,161,1000
4,tt0449088,Pirates of the Caribbean: At World's End,2007,25 May 2007,167.0,PG-13,7.1,314040375.0,7640482.0,321680857.0,...,0.0,0.0,0.0,0.0,USA,.,.,1.0149309794271015,1019.24,767.867
5,tt0974015,Justice League,2017,17 Nov 2017,121.0,PG-13,6.4,19449630.0,40475432.0,59925062.0,...,0.0,0.0,0.0,0.0,USA,Canada,UK,0.08492387237781913,375.43,745.33
6,tt2379713,Spectre,2015,06 Nov 2015,148.0,PG-13,6.8,12917812.0,24092452.0,37010264.0,...,0.0,0.0,0.0,0.0,UK,USA,Austria,0.064565114413192,472.11,755.774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2072,tt4846340,Hidden Figures,2016,06 Jan 2017,126.0,PG,7.8,18636040.0,11544851.0,30180891.0,...,0.0,0.0,0.0,0.0,USA,.,.,0.10987759034197629,590.648,1034.27
2699,tt6644200,A Quiet Place,2018,06 Apr 2018,95.0,PG-13,7.5,6628735.0,10373895.0,17002630.0,...,0.0,0.0,0.0,0.0,USA,.,.,0.035254660432006465,159,920
3807,tt5052448,Get Out,2017,24 Feb 2017,104.0,R,7.7,7363633.0,6449455.0,13813088.0,...,0.0,0.0,0.0,0.0,USA,Japan,.,0.04182915918887264,579.142,939.32
4626,tt0121766,Star Wars: Episode III - Revenge of the Sith,2005,19 May 2005,139.0,PG-13,7.5,.,.,.,...,0.0,0.0,0.0,0.0,USA,.,.,.,868.382,967.722


In [25]:
# 영화별 순수익 (contract_price를 비용으로 볼 때)
df_meta['net_profit'] = '.'
for i in df_meta.index:
    if df_meta['movie_down_sales'][i] != '.':
        df_meta.loc[i,'net_profit'] = df_meta.loc[i,'movie_down_sales'] - df_meta.loc[i,'contract_price_inf']

In [26]:
df_meta[df_meta['net_profit'] != '.']['net_profit'].sum()
print('Net Profit: ${} (2018)'.format(df_meta[df_meta['net_profit'] != '.']['net_profit'].sum()))

Net Profit: $-18424.47258914714 (2018)


In [27]:
# ROI (contract_price를 투자(즉 자산)로 볼 때)
            
total_revenue = df_meta[df_meta['movie_down_sales'] != '.']['movie_down_sales'].sum() # 원래 total_net_profit이 정의에 부합하나 혼동 방지를 위해 revenue로 대체
total_investment = df_meta[df_meta['contract_price_inf'] != '.']['contract_price_inf'].sum()

ROI = float(total_revenue/total_investment)*100
print('ROI: %.2f%%'%ROI)


ROI: 79.45%


In [28]:
total_investment

89669.487685054

In [29]:
total_revenue

71245.0150959068

In [43]:
df_meta['movie_down_sales']

0             .
1             .
2       1042.26
3           161
4       1019.24
         ...   
5148          .
5149          .
5150          .
5151          .
5152          .
Name: movie_down_sales, Length: 5153, dtype: object

In [47]:
df_meta['movie_down_sales'] = pd.to_numeric(df_meta['movie_down_sales'], errors='coerce')
           
df_inv = df_meta[df_meta['inv_exist']==1]

### 연간 ROI

In [62]:
duration = pd.Series([5,4,3,2,1], index=[2014.0, 2015.0, 2016.0, 2017.0,  2018.0])

In [48]:
# 판권 계약연도별 누적 매출
df_inv.groupby('contract_year').agg('sum')['movie_down_sales']

contract_year
2014.0    34442.405145
2015.0    20818.749137
2016.0     9006.588815
2017.0     5019.072000
2018.0     1958.200000
Name: movie_down_sales, dtype: float64

In [63]:
# 판권 계약연도별 연평균 매출
df_inv.groupby('contract_year').agg('sum')['movie_down_sales'] / duration

contract_year
2014.0    6888.481029
2015.0    5204.687284
2016.0    3002.196272
2017.0    2509.536000
2018.0    1958.200000
dtype: float64

In [50]:
# 판권 계약 연도별 투자
df_inv.groupby('contract_year').agg('sum')['contract_price']

contract_year
2014.0    35140
2015.0    15610
2016.0    11370
2017.0     9110
2018.0     9420
Name: contract_price, dtype: int64

In [51]:
df_inv.groupby('contract_year').count()['contract_price']

contract_year
2014.0    45
2015.0    22
2016.0    15
2017.0    12
2018.0    12
Name: contract_price, dtype: int64

In [64]:
# 판권 계약 연도별 ROI
(df_inv.groupby('contract_year').agg('sum')['movie_down_sales'] / duration) / df_inv.groupby('contract_year').agg('sum')['contract_price']

contract_year
2014.0    0.196030
2015.0    0.333420
2016.0    0.264045
2017.0    0.275470
2018.0    0.207877
dtype: float64

# 연간 매출 계산

In [30]:
df_download.groupby('down_year').agg('sum').loc[:,'down_price']

down_year
2014     5018.313916
2015    12192.782695
2016    14967.957485
2017    20472.071000
2018    18593.890000
Name: down_price, dtype: float64

In [31]:
df_customer

,customer_id,gender,state,age,age_group,married,kids_under12,register_year,drop_flag,drop_year,last_down_date,down_flag,customer_sales
0,C5001,F,NC,51,50s,M,N,2014,Y,2015,12-23-2014,1,10.9973
1,C5002,M,CA,23,20s,S,N,2014,N,.,10-25-2016,1,23.6616
2,C5003,F,TX,39,30s,M,N,2014,N,.,12-21-2018,1,48.8217
3,C5004,F,TX,69,60s+,M,N,2014,N,.,12-30-2014,1,38.8227
4,C5005,F,SC,60,60s+,M,N,2014,N,.,12-09-2018,1,36.842
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272,C8273,F,SC,49,40s,M,N,2018,N,.,12-18-2018,1,9.99
3273,C8274,F,NV,61,60s+,M,N,2018,N,.,12-29-2018,1,21.77
3274,C8275,F,CA,59,50s,S,N,2018,N,.,11-23-2018,1,4.41
3275,C8276,F,TX,36,30s,S,Y,2018,N,.,12-05-2018,1,13.48


In [32]:
df_customer.groupby('register_year').count()

,customer_id,gender,state,age,age_group,married,kids_under12,drop_flag,drop_year,last_down_date,down_flag,customer_sales
register_year,,,,,,,,,,,,
2014,1017,1017,1017,1017,1017,1017,1017,1017,1017,1017,1017,1017
2015,728,728,728,728,728,728,728,728,728,728,728,728
2016,662,662,662,662,662,662,662,662,662,662,662,662
2017,861,861,861,861,861,861,861,861,861,861,861,861
2018,9,9,9,9,9,9,9,9,9,9,9,9


In [33]:
df_customer.groupby('drop_year').count()

,customer_id,gender,state,age,age_group,married,kids_under12,register_year,drop_flag,last_down_date,down_flag,customer_sales
drop_year,,,,,,,,,,,,
.,2618,2618,2618,2618,2618,2618,2618,2618,2618,2618,2618,2618
2015,130,130,130,130,130,130,130,130,130,130,130,130
2016,143,143,143,143,143,143,143,143,143,143,143,143
2017,189,189,189,189,189,189,189,189,189,189,189,189
2018,197,197,197,197,197,197,197,197,197,197,197,197
